In [0]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import fashion_mnist,mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from scipy.stats import pearsonr
from tqdm import tqdm
from scipy import ndimage
from IPython.display import clear_output

In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
NUM_CHANNELS = 1
MODEL_NAME = "MNIST_data_augmentation"
PATH = ""
NR_OF_RUNS = 10

# Preprocess

In [0]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [56]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_trainc = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_full = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_full /= 255
x_test /= 255

# Model

In [0]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

#Predict


In [0]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y,voting = 'hard'):
    
    if voting == "soft":
      prediction = soft_voting(models, X)
    elif voting == "hard":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")

    return accuracy_score(prediction, np.argmax(Y, axis=1))

#Augmentation 


In [0]:
def rotate_image(image):
    if np.random.rand() < 0.2:
        angles = np.linspace(1,10,10)
        rotation_angle = np.random.choice(angles)
        if np.random.rand() < 0.5:
            image = ndimage.rotate(image, rotation_angle, reshape = False)
        else:
            image = ndimage.rotate(image, -rotation_angle, reshape = False) 
    return image

In [0]:
def horizontal_shift(image):
    if np.random.rand() < 0.2:
        num_rows, num_cols = image.shape[:2]
        if np.random.rand() < 0.5:
            translation_matrix = np.float32([ [1,0,3], [0,1,0] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
        else:
            translation_matrix = np.float32([ [1,0,-3], [0,1,0] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
    return image

In [0]:
def vertical_shift(image):
    if np.random.rand() < 0.2:
        num_rows, num_cols = image.shape[:2]
        if np.random.rand() < 0.5:
            translation_matrix = np.float32([ [1,0,0], [0,1,2] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
        else:
            translation_matrix = np.float32([ [1,0,0], [0,1,-2] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
    return image

In [0]:
funcs = [rotate_image,
         horizontal_shift, 
         vertical_shift]

In [0]:
augmentations = ["rotation", "horizontal shifting", "vertical shifting"]

# Train

In [15]:
for run in range(1, NR_OF_RUNS+1):
    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_trainc, test_size=0.20, shuffle= True)
    
    models = []
    accuracies = []
    predictions = []
    for i in range(len(funcs)):
        print(f"\n ===== Train model: Data augmentation method: {augmentations[i]}  =====")
        
        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)
        
        #data augmentation function
        preprocessing_function = funcs[i]

        #data augmentation generator
        datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)
        datagen = datagen.flow(x_train,y_train, batch_size = BATCH_SIZE)
        
        # weight init method
        model = MNISTmodel(IMAGE_SIZE,NUM_CLASSES,NUM_CHANNELS)
    
        #save weights 
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{augmentations[i]}.h5"
            
        #save weights 
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{augmentations[i]}.h5"
        if os.path.exists(weights_path):
            print(f"Skipping training of model {augmentations[i]}: weights exists")
            model.load_weights(weights_path)
        else:
            # initiate early stopping
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{augmentations[i]}.csv", separator=';')
            #train
            model.fit_generator(datagen,
                                epochs = EPOCHS,
                                validation_data = (x_val,y_val),
                                shuffle = True,
                                callbacks=[es, csv_logger])
            model.save_weights(weights_path)
            
        models.append(model)
        y_prob = model.predict(x_test) 
        predictions.append(y_prob.argmax(axis=-1))
        acc = model.evaluate(x_test,y_testc)[1]
        accuracies.append(acc)

        print(f"Model: {augmentations[i]} added. Resulting score: {acc}")

    # Results  
        
    # Accuracy vs data augmentation method
        
    print("\n ===== Accuracy vs weight init methods =====")
    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df["data_augmentaion_method"] = augmentations
    display(accuracy_df)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy.csv")
        
    print("\n ===== Converting Binary classification =====")
    classified = []
    for prediction in tqdm(predictions):
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
        
    ## Correlation between models
    print("\n ===== Correlation =====")  
    correlation_matrix = []

    for ix, x in enumerate(classified):
        row = []
  
        for iy, y in enumerate(classified):
            if (ix == iy):
                row.append(np.nan)
            else:
                row.append(pearsonr(x,y)[0])

        correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.columns = augmentations
    correlation_matrix_df.index = augmentations
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix.csv")
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))
    
    
    print("\n ===== Computing ensemble accuracy =====")  
    # Ensemble accuracy
    accuracy_hard = predict(models, x_test, y_testc,voting = 'hard')
    print("Accuracy of ensemble using hard voting: " + str(accuracy_hard))
    accuracy_soft = predict(models, x_test, y_testc,voting = 'soft')
    print("Accuracy of ensemble using soft voting: " + str(accuracy_soft))
    
    
    print("\n ===== Computing ensemble accuracy =====")  
    # Save the results
    file = PATH + MODEL_NAME + f"/results_.csv"
    df = pd.DataFrame([[run,correlation,accuracy_hard,accuracy_soft]])

    if not os.path.isfile(file):
        df.to_csv(file, header=["run", "correlation","accuracy_hard_voting","accuracy_soft_voting"], index=False)
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False)

    clear_output(wait=True)


 ===== Train model: Data augmentation method: rotation  =====
Epoch 1/9999
375/375 [==============================] - 6s 17ms/step - loss: 2.2834 - accuracy: 0.1497 - val_loss: 2.2683 - val_accuracy: 0.1209
Epoch 2/9999
375/375 [==============================] - 6s 16ms/step - loss: 2.1663 - accuracy: 0.2408 - val_loss: 2.1167 - val_accuracy: 0.3043
Epoch 3/9999
375/375 [==============================] - 6s 16ms/step - loss: 2.0438 - accuracy: 0.3343 - val_loss: 1.9581 - val_accuracy: 0.3593
Epoch 4/9999
375/375 [==============================] - 6s 16ms/step - loss: 1.9028 - accuracy: 0.3672 - val_loss: 1.8268 - val_accuracy: 0.3891
Epoch 5/9999
375/375 [==============================] - 6s 16ms/step - loss: 1.7897 - accuracy: 0.3965 - val_loss: 1.7257 - val_accuracy: 0.4151
Epoch 6/9999
375/375 [==============================] - 6s 16ms/step - loss: 1.6984 - accuracy: 0.4298 - val_loss: 1.6450 - val_accuracy: 0.4597
Epoch 7/9999
375/375 [==============================] - 6s 16ms/ste

,Accuracy,data_augmentaion_method
0,0.7721,rotation
1,0.7656,horizontal shifting
2,0.7500,vetical shifting


100%|██████████| 3/3 [00:00<00:00, 317.74it/s]


 ===== Converting Binary classification =====

 ===== Correlation =====


,rotation,horizontal shifting,vetical shifting
rotation,NaN,0.622847,0.633811
horizontal shifting,0.622847,NaN,0.620931
vetical shifting,0.633811,0.620931,NaN


Average correlation: 0.6258631391978263

 ===== Computing ensemble accuracy =====
Accuracy of ensemble using hard voting: 0.7824
Accuracy of ensemble using soft voting: 0.788

 ===== Computing ensemble accuracy =====


# Results

In [0]:
from scipy import stats

In [0]:
baseline = [0.7622,
            0.7632,
            0.7672,
            0.7498,
            0.7658,
            0.7454,
            0.7652,
            0.7777,
            0.753,
            0.7692]

In [45]:
augmented = pd.read_csv("/content/MNIST_data_augmentation/results_.csv")
augmented = augmented.values
augmented

array([[ 1.        ,  0.62610946,  0.7918    ,  0.7978    ],
       [ 2.        ,  0.63458471,  0.7861    ,  0.7918    ],
       [ 3.        ,  0.6107443 ,  0.7699    ,  0.7793    ],
       [ 4.        ,  0.61110583,  0.7662    ,  0.774     ],
       [ 5.        ,  0.6074457 ,  0.7755    ,  0.7804    ],
       [ 6.        ,  0.63162062,  0.7803    ,  0.7862    ],
       [ 7.        ,  0.63629137,  0.7776    ,  0.7846    ],
       [ 8.        ,  0.61766347,  0.7746    ,  0.7813    ],
       [ 9.        ,  0.63189894,  0.777     ,  0.7862    ],
       [10.        ,  0.62586314,  0.7824    ,  0.788     ]])

In [0]:
augmented_hard = augmented[:,2]
augmented_soft = augmented[:,3]

In [47]:
np.mean(augmented_hard)

0.7781399999999999

In [48]:
np.mean(augmented_soft)

0.7849600000000001

In [49]:
np.mean(baseline)

0.7618699999999999

In [50]:
stats.ttest_rel(augmented_hard,baseline)

Ttest_relResult(statistic=4.356297917481842, pvalue=0.001833334328602048)

In [51]:
stats.ttest_rel(augmented_soft,baseline)

Ttest_relResult(statistic=6.3006869583625695, pvalue=0.0001408690514115729)

## Accuracy
The final accuracy of the ensamble on the test set

In [52]:
!zip -r /content/MNIST_data_augmentation.zip /content/MNIST_data_augmentation

  adding: content/MNIST_data_augmentation/ (stored 0%)
  adding: content/MNIST_data_augmentation/9/ (stored 0%)
  adding: content/MNIST_data_augmentation/9/correlation_matrix.csv (deflated 50%)
  adding: content/MNIST_data_augmentation/9/accuracy.csv (deflated 19%)
  adding: content/MNIST_data_augmentation/9/weights/ (stored 0%)
  adding: content/MNIST_data_augmentation/9/weights/weights-vetical shifting.h5 (deflated 71%)
  adding: content/MNIST_data_augmentation/9/weights/weights-horizontal shifting.h5 (deflated 71%)
  adding: content/MNIST_data_augmentation/9/weights/weights-rotation.h5 (deflated 71%)
  adding: content/MNIST_data_augmentation/9/history/ (stored 0%)
  adding: content/MNIST_data_augmentation/9/history/history-vetical shifting.csv (deflated 53%)
  adding: content/MNIST_data_augmentation/9/history/history-horizontal shifting.csv (deflated 52%)
  adding: content/MNIST_data_augmentation/9/history/history-rotation.csv (deflated 52%)
  adding: content/MNIST_data_augmentation